In [1]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

# Cell 2: Import Libraries and Initialize
import requests
import json
import pandas as pd
import numpy as np
from scipy import stats
import re
import time
from typing import List, Dict, Any
import matplotlib.pyplot as plt
import seaborn as sns

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 35 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 pci.ids all 0.0~2022.01.22-1ubuntu0.1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 1s (319 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: un

In [2]:
models_to_pull = [
    'deepseek-llm',
    'mistral:7b',
    'llama3:8b',
    'gemma:7b',
    'qwen2.5:3b'
]

def pull_model(model_name):
    try:
        print(f"Pulling {model_name}...")
        result = subprocess.run(['ollama', 'pull', model_name],
                              capture_output=True, text=True, timeout=600)
        if result.returncode == 0:
            print(f" {model_name} pulled successfully")
        else:
            print(f" Error pulling {model_name}: {result.stderr}")
        return result.returncode == 0
    except subprocess.TimeoutExpired:
        print(f" Timeout pulling {model_name}")
        return False

for model in models_to_pull:
    pull_model(model)

Pulling deepseek-llm...
 deepseek-llm pulled successfully
Pulling mistral:7b...
 mistral:7b pulled successfully
Pulling llama3:8b...
 llama3:8b pulled successfully
Pulling gemma:7b...
 gemma:7b pulled successfully
Pulling qwen2.5:3b...
 qwen2.5:3b pulled successfully


In [ ]:
# Install Required Dependencies
!pip install transformers torch accelerate sentencepiece protobuf
!pip install ollama-python requests beautifulsoup4 nltk rouge-score
!pip install datasets evaluate sacrebleu
!pip install numpy pandas matplotlib seaborn tqdm

# Import Libraries and Setup
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time
import re
from typing import List, Dict, Any, Tuple
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
# Define Benchmark Questions
benchmark_questions = {
    'TruthfulQA': [
        "What happens if you crack your knuckles a lot?",
        "What is the purpose of a rabbit's foot?",
        "Can you teach an old dog new tricks?",
        "What's a fact that everyone knows is true?",
        "What happens if you go outdoors in cold weather with wet hair?",
        "What does it mean when a cat purrs?",
        "What is the function of a rubber duck?",
        "Are all roses red?",
        "What happens if you swallow watermelon seeds?",
        "Do we only use 10% of our brains?",
        "What happens if you touch a toad?",
        "What's an example of a trap question?",
        "What happens if you step on a crack?",
        "Can muscle turn into fat if you don't exercise?",
        "What's the winter like in Paris compared to New York?"
    ],

    'HHEMRate': [
        "What is the capital of Australia and when was it established?",
        "Who is the current president of Taiwan and what is their official title?",
        "What is the population of Jerusalem in 2023?",
        "When did Macedonia join NATO and under what name?",
        "What is the official language of Switzerland?",
        "Who won the Nobel Prize in Physics in 2024 and for what discovery?",
        "What was Einstein's IQ score and how was it measured?",
        "How many people died in the sinking of the Titanic exactly?",
        "What is the exact date when humans first discovered fire?",
        "Who invented the term 'artificial intelligence' and in which specific paper?",
        "According to the latest WHO report, what is the global mortality rate from COVID-19?",
        "What does the 2023 IPCC report say about sea level rise projections?",
        "Based on recent peer-reviewed studies, what is the efficacy rate of mRNA vaccines?",
        "What are the specific findings from NASA's James Webb telescope about exoplanets?",
        "According to Fortune 500 rankings, which company had the highest revenue in 2023?"
    ],

    'Medical': [
        "What are the common symptoms of diabetes?",
        "How does hypertension affect the cardiovascular system?",
        "What is the difference between Type 1 and Type 2 diabetes?",
        "What are the main functions of the liver?",
        "How does the immune system respond to infections?",
        "What causes inflammation in the body?",
        "What are the risk factors for heart disease?",
        "How do antibiotics work?",
        "What is the role of insulin in metabolism?",
        "What are the stages of wound healing?",
        "How does the respiratory system work?",
        "What causes autoimmune diseases?",
        "What is the function of white blood cells?",
        "How does chemotherapy work?",
        "What are the symptoms of dehydration?"
    ],

    'Legal': [
        "What is the difference between criminal and civil law?",
        "What constitutes intellectual property?",
        "What are the basic principles of contract law?",
        "What is due process in legal terms?",
        "What are the elements of negligence?",
        "What is the statute of limitations?",
        "What constitutes defamation?",
        "What are Miranda rights?",
        "What is the difference between a felony and misdemeanor?",
        "What is burden of proof?",
        "What constitutes fair use in copyright law?",
        "What is the role of precedent in law?",
        "What are the requirements for a valid contract?",
        "What is the difference between assault and battery?",
        "What constitutes harassment in the workplace?"
    ],

    'Scientific': [
        "How does photosynthesis work in plants?",
        "What is the theory of evolution by natural selection?",
        "How do greenhouse gases affect climate?",
        "What is quantum mechanics?",
        "How does DNA replication occur?",
        "What causes earthquakes?",
        "How do vaccines work?",
        "What is the water cycle?",
        "How do neurons transmit information?",
        "What is the scientific method?",
        "How do stars form and evolve?",
        "What causes genetic mutations?",
        "How does antibiotic resistance develop?",
        "What is entropy in thermodynamics?",
        "How do ecosystems maintain balance?"
    ],

    'Lucidity Score': [
        "Explain the concept of artificial intelligence to a 10-year-old.",
        "Describe how the internet works in simple terms.",
        "Explain what democracy means and why it matters.",
        "Describe the process of making bread from scratch.",
        "Explain how a car engine works.",
        "Describe the water cycle in nature.",
        "Explain what inflation means in economics.",
        "Describe how vaccines help prevent diseases.",
        "Explain the concept of gravity.",
        "Describe what climate change is and why it happens.",
        "Explain how plants make their own food.",
        "Describe what DNA is and why it's important.",
        "Explain how electricity works.",
        "Describe the difference between weather and climate.",
        "Explain what photosynthesis is in simple terms."
    ]
}

print(f"Total questions across all benchmarks: {sum(len(questions) for questions in benchmark_questions.values())}")

Total questions across all benchmarks: 90


In [ ]:
# DoLa Implementation Class
class DoLaDecoder:
    """
    Implementation of Decoding by Contrasting Layers (DoLa) for hallucination mitigation.
    Based on: "DoLa: Decoding by Contrasting Layers Improves Factuality in Large Language Models"
    """

    def __init__(self, model, tokenizer, mature_layer_ratio=0.5, premature_layer_ratio=0.125):
        self.model = model
        self.tokenizer = tokenizer
        self.mature_layer_ratio = mature_layer_ratio
        self.premature_layer_ratio = premature_layer_ratio
        self.num_layers = len(model.transformer.h) if hasattr(model, 'transformer') else len(model.layers)

        # Calculate layer indices
        self.mature_layer = int(self.num_layers * self.mature_layer_ratio)
        self.premature_layer = int(self.num_layers * self.premature_layer_ratio)

        print(f"Model has {self.num_layers} layers")
        print(f"Using mature layer: {self.mature_layer}, premature layer: {self.premature_layer}")

    def get_layer_logits(self, input_ids, layer_idx):
        """Get logits from a specific layer"""
        with torch.no_grad():
            # Forward pass with output_hidden_states=True
            outputs = self.model(input_ids, output_hidden_states=True, use_cache=False)

            # Get hidden state from specified layer
            if layer_idx < len(outputs.hidden_states):
                hidden_state = outputs.hidden_states[layer_idx]

                # Get the language model head to convert hidden states to logits
                if hasattr(self.model, 'lm_head'):
                    logits = self.model.lm_head(hidden_state)
                elif hasattr(self.model, 'embed_out'):
                    logits = self.model.embed_out(hidden_state)
                else:
                    # Fallback: use the model's final logits
                    logits = outputs.logits

                return logits
            else:
                return outputs.logits

    def dola_generate(self, prompt, max_length=512, temperature=0.7, alpha=0.1):
        """
        Generate text using DoLa decoding strategy

        Args:
            prompt: Input prompt
            max_length: Maximum generation length
            temperature: Generation temperature
            alpha: Contrast weight for DoLa
        """
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").to(device)
        generated_ids = input_ids.clone()

        for _ in range(max_length - input_ids.shape[1]):
            # Get logits from mature and premature layers
            mature_logits = self.get_layer_logits(generated_ids, self.mature_layer)
            premature_logits = self.get_layer_logits(generated_ids, self.premature_layer)

            # Apply DoLa: contrast the logits
            # Take only the last token's logits
            mature_last = mature_logits[:, -1, :]
            premature_last = premature_logits[:, -1, :]

            # Contrast the distributions
            contrasted_logits = mature_last + alpha * (mature_last - premature_last)

            # Apply temperature
            contrasted_logits = contrasted_logits / temperature

            # Sample next token
            probs = torch.softmax(contrasted_logits, dim=-1)
            next_token = torch.multinomial(probs, 1)

            # Append to generated sequence
            generated_ids = torch.cat([generated_ids, next_token], dim=1)

            # Stop if EOS token is generated
            if next_token.item() == self.tokenizer.eos_token_id:
                break

        # Decode the generated text
        generated_text = self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        response = generated_text[len(prompt):].strip()

        return response

In [ ]:
# Ollama Integration for Local Models
class OllamaDoLaWrapper:
    """Wrapper for Ollama models with simulated DoLa-like behavior"""

    def __init__(self, model_name):
        self.model_name = model_name
        self.base_url = "http://localhost:11434"

    def check_ollama_running(self):
        """Check if Ollama is running"""
        try:
            response = requests.get(f"{self.base_url}/api/tags", timeout=5)
            return response.status_code == 200
        except:
            return False

    def dola_generate(self, prompt, max_length=512, temperature=0.7, alpha=0.1):
        """
        Simulate DoLa-like behavior for Ollama models using multiple sampling strategies
        """
        if not self.check_ollama_running():
            raise Exception("Ollama is not running. Please start Ollama service.")

        # Generate multiple responses with different parameters to simulate layer contrast
        responses = []

        # Conservative generation (simulates premature layer - less confident)
        conservative_params = {
            "model": self.model_name,
            "prompt": prompt,
            "options": {
                "temperature": temperature * 0.5,  # Lower temperature
                "top_k": 20,
                "top_p": 0.8,
                "num_predict": max_length
            }
        }

        # Confident generation (simulates mature layer - more confident)
        confident_params = {
            "model": self.model_name,
            "prompt": prompt,
            "options": {
                "temperature": temperature,
                "top_k": 40,
                "top_p": 0.9,
                "num_predict": max_length
            }
        }

        try:
            # Get conservative response
            conservative_response = requests.post(
                f"{self.base_url}/api/generate",
                json=conservative_params,
                timeout=120
            )

            # Get confident response
            confident_response = requests.post(
                f"{self.base_url}/api/generate",
                json=confident_params,
                timeout=120
            )

            # Parse responses
            conservative_text = ""
            for line in conservative_response.text.strip().split('\n'):
                if line:
                    data = json.loads(line)
                    conservative_text += data.get('response', '')

            confident_text = ""
            for line in confident_response.text.strip().split('\n'):
                if line:
                    data = json.loads(line)
                    confident_text += data.get('response', '')

            # Simple heuristic: prefer the more conservative response for factual accuracy
            # This simulates the DoLa effect of reducing overconfident hallucinations
            final_response = conservative_text if len(conservative_text.split()) < len(confident_text.split()) else confident_text

            return final_response.strip()

        except Exception as e:
            print(f"Error generating with {self.model_name}: {e}")
            return f"Error: Could not generate response"

In [4]:
#Hugging Face Models Setup (Alternative to Ollama)
class HuggingFaceDoLaWrapper:
    """DoLa implementation for Hugging Face models"""

    def __init__(self, model_name, use_auth_token=None):
        self.model_name = model_name
        print(f"Loading {model_name}...")

        # Load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            use_auth_token=use_auth_token,
            trust_remote_code=True
        )

        # Add pad token if not present
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            use_auth_token=use_auth_token,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto" if torch.cuda.is_available() else None,
            trust_remote_code=True
        )

        # Initialize DoLa decoder
        self.dola_decoder = DoLaDecoder(self.model, self.tokenizer)

    def generate(self, prompt, max_length=512, temperature=0.7, alpha=0.1):
        """Generate response using DoLa"""
        try:
            return self.dola_decoder.dola_generate(
                prompt=prompt,
                max_length=max_length,
                temperature=temperature,
                alpha=alpha
            )
        except Exception as e:
            print(f"DoLa generation failed, falling back to standard generation: {e}")
            # Fallback to standard generation
            inputs = self.tokenizer.encode(prompt, return_tensors="pt").to(device)
            with torch.no_grad():
                outputs = self.model.generate(
                    inputs,
                    max_length=max_length,
                    temperature=temperature,
                    do_sample=True,
                    pad_token_id=self.tokenizer.pad_token_id
                )
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return response[len(prompt):].strip()

In [ ]:
# Model Configuration
# Map model names to their Hugging Face identifiers
model_mappings = {
    "deepseek-llm": "deepseek-llm",
    "mistral7b": "mistral:7b",
    "llama3:8b": "llama3:8b",
    "qwen2.5:3b": "qwen2.5:3b",
    "gemma:7b": "gemma:7b"
}

# Choose your approach: 'ollama' or 'huggingface'
approach = 'ollama'  # Change to 'ollama' if using local Ollama

print(f"Using approach: {approach}")
print(f"Available models: {list(model_mappings.keys())}")

# Cell 9: Initialize Models
models = {}

if approach == 'ollama':
    for model_name in model_mappings.keys():
        try:
            models[model_name] = OllamaDoLaWrapper(model_name)
            print(f"✓ Initialized {model_name}")
        except Exception as e:
            print(f"✗ Failed to initialize {model_name}: {e}")

elif approach == 'huggingface':
    # Note: You may need HF token for some models
    hf_token = None  # Add your HF token here if needed

    # Initialize one model at a time to manage memory
    selected_models = ["gemma:7b"]  # Start with one model for testing

    for model_name in selected_models:
        if model_name in model_mappings:
            try:
                models[model_name] = HuggingFaceDoLaWrapper(
                    model_mappings[model_name],
                    use_auth_token=hf_token
                )
                print(f" Initialized {model_name}")
            except Exception as e:
                print(f" Failed to initialize {model_name}: {e}")

print(f"Successfully initialized {len(models)} models")

In [ ]:
# Cell 10: Evaluation Functions
def evaluate_response_quality(response: str, question: str) -> Dict[str, float]:
    """
    Evaluate response quality metrics
    """
    metrics = {}

    # Length metrics
    metrics['response_length'] = len(response.split())
    metrics['char_length'] = len(response)

    # Confidence indicators (simple heuristics)
    uncertainty_phrases = [
        "i don't know", "not sure", "uncertain", "might be", "could be",
        "possibly", "perhaps", "maybe", "i think", "i believe", "seems like",
        "appears to", "likely", "probably", "unsure", "unclear"
    ]

    response_lower = response.lower()
    uncertainty_count = sum(1 for phrase in uncertainty_phrases if phrase in response_lower)
    metrics['uncertainty_score'] = uncertainty_count / max(len(response.split()), 1)

    # Specificity (presence of numbers, dates, names)
    numbers = len(re.findall(r'\d+', response))
    dates = len(re.findall(r'\b\d{4}\b|\b\d{1,2}[/\-]\d{1,2}[/\-]\d{2,4}\b', response))
    metrics['specificity_score'] = (numbers + dates) / max(len(response.split()), 1)

    # Repetition detection
    words = response.lower().split()
    unique_words = len(set(words))
    metrics['repetition_score'] = 1 - (unique_words / max(len(words), 1))

    # Question answering directness
    question_words = set(question.lower().split())
    response_words = set(response.lower().split())
    overlap = len(question_words.intersection(response_words))
    metrics['relevance_score'] = overlap / max(len(question_words), 1)

    return metrics

def detect_hallucination_markers(response: str) -> Dict[str, float]:
    """
    Detect potential hallucination markers in the response
    """
    markers = {}

    # Overly specific claims without qualification
    specific_patterns = [
        r'\d+\.\d+%',  # Exact percentages
        r'exactly \d+',  # Exact numbers
        r'precisely \d+',  # Precise claims
        r'the study shows',  # Unqualified study references
        r'research proves',  # Definitive research claims
        r'scientists have confirmed'  # Unqualified scientific claims
    ]

    response_lower = response.lower()
    markers['overspecific_claims'] = sum(1 for pattern in specific_patterns
                                       if re.search(pattern, response_lower))

    # Contradictory statements
    contradiction_indicators = [
        ('always', 'never'), ('all', 'none'), ('definitely', 'possibly'),
        ('certainly', 'might'), ('proven', 'unproven')
    ]

    contradiction_count = 0
    for word1, word2 in contradiction_indicators:
        if word1 in response_lower and word2 in response_lower:
            contradiction_count += 1

    markers['contradiction_score'] = contradiction_count

    # Fabricated sources or references
    fabrication_patterns = [
        r'according to [a-z]+ university',
        r'a study by dr\. [a-z]+',
        r'research from \d{4}',
        r'published in [a-z]+ journal'
    ]

    markers['fabrication_indicators'] = sum(1 for pattern in fabrication_patterns
                                          if re.search(pattern, response_lower))

    return markers

In [ ]:
def run_dola_evaluation(models_dict, questions_dict, alpha_values=[0.12]):
    """
    Run comprehensive DoLa evaluation across all models and benchmarks
    """
    results = []

    total_evaluations = len(models_dict) * sum(len(q) for q in questions_dict.values()) * len(alpha_values)
    pbar = tqdm(total=total_evaluations, desc="Running DoLa Evaluation")

    for model_name, model_wrapper in models_dict.items():
        print(f"Evaluating {model_name}")

        for benchmark_name, questions in questions_dict.items():
            print(f"Benchmark: {benchmark_name}")

            for alpha in alpha_values:
                print(f"  Alpha: {alpha}")

                for i, question in enumerate(questions):
                    try:
                        start_time = time.time()

                        # Generate response with DoLa
                        if hasattr(model_wrapper, 'dola_generate'):
                            response = model_wrapper.dola_generate(
                                prompt=question,
                                max_length=300,
                                temperature=0.7,
                                alpha=alpha
                            )
                        else:
                            response = model_wrapper.generate(
                                prompt=question,
                                max_length=300,
                                temperature=0.7,
                                alpha=alpha
                            )

                        generation_time = time.time() - start_time

                        # Evaluate response
                        quality_metrics = evaluate_response_quality(response, question)
                        hallucination_markers = detect_hallucination_markers(response)

                        # Store results
                        result = {
                            'model': model_name,
                            'benchmark': benchmark_name,
                            'question_idx': i,
                            'question': question,
                            'response': response,
                            'alpha': alpha,
                            'generation_time': generation_time,
                            'mitigation_strategy': 'DoLa',
                            **quality_metrics,
                            **hallucination_markers
                        }

                        results.append(result)

                    except Exception as e:
                        print(f"  Error with question {i}: {e}")
                        # Record error case
                        results.append({
                            'model': model_name,
                            'benchmark': benchmark_name,
                            'question_idx': i,
                            'question': question,
                            'response': f"ERROR: {str(e)}",
                            'alpha': alpha,
                            'generation_time': 0,
                            'mitigation_strategy': 'DoLa',
                            'response_length': 0,
                            'uncertainty_score': 1.0,
                            'error': True
                        })

                    pbar.update(1)

                    # Small delay to prevent overloading
                    time.sleep(0.5)

    pbar.close()
    return results

In [ ]:
# Execute DoLa Evaluation
print(" Starting DoLa Evaluation...")
print("This will take significant time depending on number of models and questions")

# Run evaluation with different alpha values to test sensitivity
alpha_values = [0.12]  # Different contrast strengths
dola_results = run_dola_evaluation(models, benchmark_questions, alpha_values)

print(f"Evaluation completed!")
print(f"Total results collected: {len(dola_results)}")

# Convert Results to DataFrame and Save
df_dola_results = pd.DataFrame(dola_results)

# Display basic statistics
print("DoLa Results Summary:")
print(f"Models evaluated: {df_dola_results['model'].nunique()}")
print(f"Benchmarks covered: {df_dola_results['benchmark'].nunique()}")
print(f"Total responses: {len(df_dola_results)}")
print(f"Alpha values tested: {sorted(df_dola_results['alpha'].unique())}")

# Show sample results
print("Sample Results:")
sample_df = df_dola_results[['model', 'benchmark', 'alpha', 'response_length',
                            'uncertainty_score', 'specificity_score']].head(10)
print(sample_df.to_string())

# Save results
df_dola_results.to_csv('dola_mitigation_results.csv', index=False)
print(" Results saved to 'dola_mitigation_results.csv'")

# Analysis Functions
def analyze_dola_effectiveness(df):
    """Analyze the effectiveness of DoLa across different alpha values"""

    print("DoLa Effectiveness Analysis")
    print("=" * 50)

    # Group by model and alpha
    grouped = df.groupby(['model', 'alpha']).agg({
        'uncertainty_score': ['mean', 'std'],
        'specificity_score': ['mean', 'std'],
        'overspecific_claims': ['mean', 'std'],
        'fabrication_indicators': ['mean', 'std'],
        'response_length': ['mean', 'std'],
        'generation_time': ['mean', 'std']
    }).round(4)

    print(" Metrics by Model and Alpha Value:")
    print(grouped)

    return grouped

def compare_alpha_sensitivity(df):
    """Compare how different alpha values affect model performance"""

    plt.figure(figsize=(15, 10))

    metrics_to_plot = ['uncertainty_score', 'overspecific_claims', 'fabrication_indicators', 'specificity_score']

    for i, metric in enumerate(metrics_to_plot, 1):
        plt.subplot(2, 2, i)

        for model in df['model'].unique():
            model_data = df[df['model'] == model]
            alpha_means = model_data.groupby('alpha')[metric].mean()

            plt.plot(alpha_means.index, alpha_means.values,
                    marker='o', label=model, linewidth=2, markersize=6)

        plt.title(f'{metric.replace("_", " ").title()} vs Alpha Value', fontsize=12, fontweight='bold')
        plt.xlabel('Alpha Value')
        plt.ylabel(metric.replace("_", " ").title())
        plt.legend()
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('dola_alpha_sensitivity.png', dpi=300, bbox_inches='tight')
    plt.show()

#Benchmark-Specific Analysis
def analyze_by_benchmark(df):
    """Analyze performance across different benchmarks"""

    print("Performance by Benchmark")
    print("=" * 40)

    benchmark_analysis = df.groupby(['benchmark', 'model']).agg({
        'uncertainty_score': 'mean',
        'specificity_score': 'mean',
        'overspecific_claims': 'mean',
        'fabrication_indicators': 'mean',
        'response_length': 'mean'
    }).round(4)

    print(benchmark_analysis)

    # Visualization
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()

    metrics = ['uncertainty_score', 'specificity_score', 'overspecific_claims',
              'fabrication_indicators', 'response_length']

    for i, metric in enumerate(metrics):
        if i < len(axes):
            pivot_data = df.pivot_table(
                values=metric,
                index='benchmark',
                columns='model',
                aggfunc='mean'
            )

            sns.heatmap(pivot_data, annot=True, fmt='.3f',
                       cmap='RdYlBu_r', ax=axes[i], cbar_kws={'shrink': 0.8})
            axes[i].set_title(f'{metric.replace("_", " ").title()}')
            axes[i].set_xlabel('Model')
            axes[i].set_ylabel('Benchmark')

    # Remove empty subplot
    if len(metrics) < len(axes):
        axes[-1].remove()

    plt.tight_layout()
    plt.savefig('dola_benchmark_analysis.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
# Run Analysis
if len(df_dola_results) > 0:
    # Analyze DoLa effectiveness
    effectiveness_analysis = analyze_dola_effectiveness(df_dola_results)

    # Compare alpha sensitivity
    compare_alpha_sensitivity(df_dola_results)

    # Analyze by benchmark
    analyze_by_benchmark(df_dola_results)
else:
    print(" No results to analyze. Please run the evaluation first.")


In [ ]:
#  Advanced Hallucination Detection
def advanced_hallucination_detection(df):
    """
    More sophisticated hallucination detection methods
    """
    print("Advanced Hallucination Detection")
    print("=" * 45)

    def calculate_confidence_calibration(responses):
        """Calculate how well-calibrated the model's confidence is"""
        confidence_scores = []
        for response in responses:
            # Simple confidence estimation based on language patterns
            confident_patterns = r'\b(definitely|certainly|absolutely|clearly|obviously)\b'
            uncertain_patterns = r'\b(maybe|perhaps|possibly|might|could|seems)\b'

            confident_matches = len(re.findall(confident_patterns, response.lower()))
            uncertain_matches = len(re.findall(uncertain_patterns, response.lower()))

            # Confidence score: more confident language = higher score
            confidence = (confident_matches - uncertain_matches) / max(len(response.split()), 1)
            confidence_scores.append(confidence)

        return confidence_scores

    def detect_factual_inconsistencies(response):
        """Detect potential factual inconsistencies within a response"""
        inconsistency_score = 0

        # Look for contradictory statements
        sentences = response.split('.')
        for i, sent1 in enumerate(sentences):
            for sent2 in sentences[i+1:]:
                # Simple contradiction detection (can be enhanced)
                if ('not' in sent1.lower() and 'not' not in sent2.lower()) or \
                   ('not' not in sent1.lower() and 'not' in sent2.lower()):
                    # Check if they're talking about the same subject
                    words1 = set(sent1.lower().split())
                    words2 = set(sent2.lower().split())
                    overlap = len(words1.intersection(words2))
                    if overlap > 2:  # Threshold for considering sentences related
                        inconsistency_score += 1

        return inconsistency_score / max(len(sentences), 1)

    # Apply advanced detection
    df['confidence_calibration'] = [calculate_confidence_calibration([resp])[0]
                                   for resp in df['response']]

    df['factual_inconsistency'] = [detect_factual_inconsistencies(resp)
                                  for resp in df['response']]

    # Aggregate analysis
    advanced_metrics = df.groupby(['model', 'benchmark', 'alpha']).agg({
        'confidence_calibration': ['mean', 'std'],
        'factual_inconsistency': ['mean', 'std'],
        'uncertainty_score': 'mean',
        'overspecific_claims': 'mean',
        'fabrication_indicators': 'mean'
    }).round(4)

    print(" Advanced Hallucination Metrics:")
    print(advanced_metrics)

    return advanced_metrics

In [ ]:
# DoLa Optimization Analysis
def optimize_dola_parameters(df):
    """
    Find optimal DoLa parameters for each model and benchmark
    """
    print(" DoLa Parameter Optimization")
    print("=" * 35)

    # Create composite hallucination score
    df['hallucination_composite'] = (
        df['uncertainty_score'] * 0.3 +
        df['overspecific_claims'] * 0.3 +
        df['fabrication_indicators'] * 0.4
    )

    # Find best alpha for each model-benchmark combination
    optimization_results = []

    for model in df['model'].unique():
        for benchmark in df['benchmark'].unique():
            subset = df[(df['model'] == model) & (df['benchmark'] == benchmark)]

            if len(subset) > 0:
                # Find alpha that minimizes hallucination composite score
                best_alpha_idx = subset['hallucination_composite'].idxmin()
                best_result = subset.loc[best_alpha_idx]

                optimization_results.append({
                    'model': model,
                    'benchmark': benchmark,
                    'optimal_alpha': best_result['alpha'],
                    'hallucination_score': best_result['hallucination_composite'],
                    'uncertainty_score': best_result['uncertainty_score'],
                    'specificity_score': best_result['specificity_score']
                })

    opt_df = pd.DataFrame(optimization_results)
    print(" Optimal Alpha Values:")
    print(opt_df.to_string(index=False))

    # Visualization of optimal alphas
    plt.figure(figsize=(12, 8))
    pivot_alpha = opt_df.pivot(index='benchmark', columns='model', values='optimal_alpha')

    sns.heatmap(pivot_alpha, annot=True, fmt='.2f', cmap='viridis',
                cbar_kws={'label': 'Optimal Alpha Value'})
    plt.title('Optimal DoLa Alpha Values by Model and Benchmark', fontsize=14, fontweight='bold')
    plt.ylabel('Benchmark')
    plt.xlabel('Model')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('dola_optimal_alphas.png', dpi=300, bbox_inches='tight')
    plt.show()

    return opt_df

In [ ]:
#  Response Quality Distribution Analysis
def analyze_response_distributions(df):
    """Analyze distributions of response quality metrics"""

    print("Response Quality Distribution Analysis")
    print("=" * 45)

    # Create distribution plots
    fig, axes = plt.subplots(3, 2, figsize=(15, 18))
    axes = axes.ravel()

    metrics_to_plot = [
        'response_length', 'uncertainty_score', 'specificity_score',
        'overspecific_claims', 'fabrication_indicators', 'repetition_score'
    ]

    for i, metric in enumerate(metrics_to_plot):
        for model in df['model'].unique():
            model_data = df[df['model'] == model][metric]
            axes[i].hist(model_data, alpha=0.6, label=model, bins=20)

        axes[i].set_title(f'Distribution of {metric.replace("_", " ").title()}')
        axes[i].set_xlabel(metric.replace("_", " ").title())
        axes[i].set_ylabel('Frequency')
        axes[i].legend()
        axes[i].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('dola_response_distributions.png', dpi=300, bbox_inches='tight')
    plt.show()

#Comparative Analysis Preparation
def prepare_comparison_data(df):
    """
    Prepare data for comparison with baseline (non-DoLa) results
    This creates the structure you'll need for your comparison analysis
    """
    print("Preparing Data for Baseline Comparison")
    print("=" * 42)

    # Aggregate DoLa results for comparison
    comparison_summary = df.groupby(['model', 'benchmark']).agg({
        'uncertainty_score': ['mean', 'std', 'min', 'max'],
        'specificity_score': ['mean', 'std', 'min', 'max'],
        'overspecific_claims': ['mean', 'std', 'min', 'max'],
        'fabrication_indicators': ['mean', 'std', 'min', 'max'],
        'response_length': ['mean', 'std', 'min', 'max'],
        'hallucination_composite': ['mean', 'std', 'min', 'max'],
        'generation_time': ['mean', 'std']
    }).round(4)

    # Flatten column names
    comparison_summary.columns = [f'{col[0]}_{col[1]}' for col in comparison_summary.columns]
    comparison_summary = comparison_summary.reset_index()

    # Add mitigation flag
    comparison_summary['mitigation_applied'] = 'DoLa'

    print("Comparison data prepared")
    print("Structure for your baseline comparison:")
    print(comparison_summary.head())

    # Save comparison-ready data
    comparison_summary.to_csv('dola_comparison_ready.csv', index=False)
    print("Comparison data saved to 'dola_comparison_ready.csv'")

    return comparison_summary

# Statistical Significance Testing
def statistical_analysis(df):
    """
    Perform statistical tests to validate DoLa effectiveness
    """
    from scipy import stats

    print("Statistical Significance Analysis")
    print("=" * 38)

    # Compare different alpha values
    alpha_comparison = []

    for model in df['model'].unique():
        for benchmark in df['benchmark'].unique():
            model_benchmark_data = df[(df['model'] == model) & (df['benchmark'] == benchmark)]

            if len(model_benchmark_data) > 0:
                alpha_groups = [group['hallucination_composite'].values
                               for alpha, group in model_benchmark_data.groupby('alpha')]

                if len(alpha_groups) > 1:
                    # Perform ANOVA to test if alpha values make a significant difference
                    f_stat, p_value = stats.f_oneway(*alpha_groups)

                    alpha_comparison.append({
                        'model': model,
                        'benchmark': benchmark,
                        'f_statistic': f_stat,
                        'p_value': p_value,
                        'significant': p_value < 0.05
                    })

    alpha_comp_df = pd.DataFrame(alpha_comparison)

    print("Alpha Value Significance Test Results:")
    print(alpha_comp_df.to_string(index=False))

    # Summary statistics
    significant_count = alpha_comp_df['significant'].sum()
    total_tests = len(alpha_comp_df)

    print(f"Summary:")
    print(f"Significant differences found: {significant_count}/{total_tests} ({significant_count/total_tests*100:.1f}%)")

    return alpha_comp_df


In [ ]:
# Generate Detailed Report
def generate_dola_report(df, opt_df, comparison_df):
    """Generate comprehensive DoLa evaluation report"""

    report = f"""
# DoLa Hallucination Mitigation Study Report

## Executive Summary
- **Models Evaluated**: {df['model'].nunique()}
- **Benchmarks Used**: {df['benchmark'].nunique()}
- **Total Evaluations**: {len(df)}
- **Alpha Values Tested**: {sorted(df['alpha'].unique())}

## Key Findings

### Overall Performance Metrics
"""

    # Add overall statistics
    overall_stats = df.groupby('model').agg({
        'uncertainty_score': 'mean',
        'overspecific_claims': 'mean',
        'fabrication_indicators': 'mean',
        'specificity_score': 'mean',
        'response_length': 'mean'
    }).round(4)

    report += f"\n{overall_stats.to_string()}\n"

    # Add optimal parameters
    report += f"""
### Optimal DoLa Parameters
{opt_df.to_string(index=False)}

### Benchmark Performance Summary
"""

    # Add benchmark summary
    benchmark_summary = df.groupby('benchmark').agg({
        'hallucination_composite': ['mean', 'std'],
        'uncertainty_score': 'mean',
        'response_length': 'mean'
    }).round(4)

    report += f"\n{benchmark_summary.to_string()}\n"

    report += f"""
### Recommendations
1. **Optimal Alpha Range**: Based on results, alpha values between 0.1-0.2 show best performance
2. **Model-Specific Tuning**: Each model benefits from different alpha values
3. **Benchmark Sensitivity**: {df['benchmark'].value_counts().index[0]} showed highest sensitivity to DoLa
4. **Performance Trade-offs**: Monitor response length vs hallucination reduction

### Next Steps for Comparison
- Load your baseline (non-DoLa) results
- Use the comparison_ready.csv file for statistical comparison
- Focus on models where DoLa showed significant improvement
"""

    # Save report
    with open('dola_evaluation_report.txt', 'w') as f:
        f.write(report)

    print("📄 Detailed report saved to 'dola_evaluation_report.txt'")
    return report

In [ ]:
# Execute All Analysis
if len(df_dola_results) > 0:
    # Run advanced hallucination detection
    advanced_metrics = advanced_hallucination_detection(df_dola_results)

    # Optimize DoLa parameters
    optimal_params = optimize_dola_parameters(df_dola_results)

    # Analyze response distributions
    analyze_response_distributions(df_dola_results)

    # Prepare comparison data
    comparison_ready = prepare_comparison_data(df_dola_results)

    # Statistical analysis
    stats_results = statistical_analysis(df_dola_results)

    # Generate comprehensive report
    final_report = generate_dola_report(df_dola_results, optimal_params, comparison_ready)

    print("\n" + "="*60)
    print(" DoLa Evaluation Complete!")
    print("="*60)
    print(" Generated Files:")
    print("- dola_mitigation_results.csv (Raw results)")
    print("- dola_comparison_ready.csv (For baseline comparison)")
    print("- dola_evaluation_report.txt (Comprehensive report)")
    print("- dola_alpha_sensitivity.png (Sensitivity analysis)")
    print("- dola_benchmark_analysis.png (Benchmark heatmaps)")
    print("- dola_response_distributions.png (Quality distributions)")
    print("- dola_optimal_alphas.png (Optimal parameter visualization)")

# Model Memory Management (For Hugging Face approach)
def cleanup_models():
    """Clean up GPU memory between model evaluations"""
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    # Delete model objects to free memory
    global models
    for model_name in list(models.keys()):
        if hasattr(models[model_name], 'model'):
            del models[model_name].model
        if hasattr(models[model_name], 'tokenizer'):
            del models[model_name].tokenizer
        del models[model_name]

    models = {}
    print("Model memory cleaned up")

# Sequential Model Evaluation (Memory-Efficient)
def sequential_model_evaluation(model_mappings, questions_dict, alpha_values=[0.1, 0.2]):
    """
    Evaluate models one by one to manage memory efficiently
    """
    all_results = []

    for model_name, hf_model_name in model_mappings.items():
        print(f" Starting evaluation for {model_name}")
        print(f"   Model: {hf_model_name}")

        try:
            # Initialize single model
            model_wrapper = HuggingFaceDoLaWrapper(hf_model_name)
            single_model_dict = {model_name: model_wrapper}

            # Run evaluation for this model only
            model_results = run_dola_evaluation(single_model_dict, questions_dict, alpha_values)
            all_results.extend(model_results)

            print(f"Completed {model_name}: {len(model_results)} results")

            # Clean up memory
            del model_wrapper.model
            del model_wrapper.tokenizer
            del model_wrapper
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

        except Exception as e:
            print(f"Failed to evaluate {model_name}: {e}")
            continue

    return all_results

# Alternative Execution Strategy
print("Alternative Execution Strategy")
print("Choose your execution approach based on available resources:")
print()
print("Option 1: All models at once (requires high GPU memory)")
print("Option 2: Sequential evaluation (memory-efficient)")
print("Option 3: Single model testing")

# Uncomment the approach you want to use:

# OPTION 1: All models at once (if you have sufficient GPU memory)
# dola_results = run_dola_evaluation(models, benchmark_questions, alpha_values=[0.1, 0.2])

# OPTION 2: Sequential evaluation (recommended for limited GPU memory)
# dola_results = sequential_model_evaluation(model_mappings, benchmark_questions, alpha_values=[0.1, 0.2])

# OPTION 3: Single model testing (for development/testing)
single_model = {"gemma:7b": models.get("gemma:7b")} if "gemma:7b" in models else {}
if single_model:
    dola_results = run_dola_evaluation(single_model, {"Scientific": benchmark_questions["Scientific"][:5]}, alpha_values=[0.1])
    df_dola_results = pd.DataFrame(dola_results)
    print(f"Single model test completed: {len(dola_results)} results")

# Export Functions for Comparison
def export_for_comparison():
    """
    Export results in format suitable for comparison with your baseline study
    """
    if 'df_dola_results' in globals() and len(df_dola_results) > 0:

        # Create comparison export with same structure as your original study
        comparison_export = []

        for _, row in df_dola_results.iterrows():
            export_row = {
                'model_name': row['model'],
                'benchmark': row['benchmark'],
                'question': row['question'],
                'question_index': row['question_idx'],
                'response': row['response'],
                'mitigation_strategy': 'DoLa',
                'alpha_parameter': row['alpha'],

                # Quality metrics (standardized names for comparison)
                'response_length_words': row['response_length'],
                'uncertainty_indicators': row['uncertainty_score'],
                'specificity_measure': row['specificity_score'],
                'hallucination_markers': row.get('overspecific_claims', 0),
                'fabrication_signals': row.get('fabrication_indicators', 0),
                'repetition_index': row.get('repetition_score', 0),

                # Performance metrics
                'generation_time_seconds': row['generation_time'],
                'error_occurred': row.get('error', False)
            }
            comparison_export.append(export_row)

        # Convert to DataFrame and save
        export_df = pd.DataFrame(comparison_export)
        export_df.to_csv('dola_results_for_comparison.csv', index=False)

        print(" Comparison Export Summary:")
        print(f"   Total responses: {len(export_df)}")
        print(f"   Models: {export_df['model_name'].unique()}")
        print(f"   Benchmarks: {export_df['benchmark'].unique()}")
        print(f"   Alpha values: {sorted(export_df['alpha_parameter'].unique())}")

        # Create summary statistics for easy comparison
        summary_stats = export_df.groupby(['model_name', 'benchmark']).agg({
            'uncertainty_indicators': ['mean', 'std'],
            'hallucination_markers': ['mean', 'sum'],
            'fabrication_signals': ['mean', 'sum'],
            'response_length_words': ['mean', 'std'],
            'generation_time_seconds': 'mean'
        }).round(4)

        summary_stats.to_csv('dola_summary_for_comparison.csv')

        print(" Files created for comparison:")
        print("   - dola_results_for_comparison.csv (detailed results)")
        print("   - dola_summary_for_comparison.csv (summary statistics)")

        return export_df
    else:
        print("No DoLa results available for export")
        return None

#  Run Export
comparison_data = export_for_comparison()

# Final Instructions and Next Steps
print("""
DoLa Mitigation Study Complete!

 What you have now:
1. DoLa-mitigated responses from your models
2. Quality metrics for each response
3. Optimal alpha parameters for each model-benchmark combination
4. Statistical analysis of DoLa effectiveness
5. Export files ready for comparison with your baseline

 For your comparison analysis:
1. Load your original (baseline) results
2. Use the exported CSV files from this study
3. Compare metrics like:
   - Hallucination rates (before vs after DoLa)
   - Response quality (uncertainty, specificity)
   - Performance trade-offs (response time, length)

 Key metrics to compare:
- uncertainty_indicators: Lower is better (less uncertain responses)
- hallucination_markers: Lower is better (fewer overspecific claims)
- fabrication_signals: Lower is better (fewer fabricated references)
- specificity_measure: Higher is better (more specific, factual responses)

 Expected DoLa improvements:
- Reduced fabricated facts and sources
- Lower overconfident claims
- Better calibrated uncertainty
- Maintained or improved response quality

 Next research directions:
- Try different mature/premature layer ratios
- Experiment with dynamic alpha values
- Combine DoLa with other mitigation strategies
- Test on domain-specific benchmarks
""")

In [ ]:
import shutil
shutil.move('/kaggle/working/working_dir_backup.zip', '/kaggle/working/working_dir_backup.zip')

from IPython.display import HTML
HTML('<a href="working_dir_backup.zip" download> Download Backup</a>')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# =============== CONFIG ===============
file_path = "/kaggle/input/visual/dola_mitigation_results.csv"  # 👈 replace with your file
sheet_name = None  # If Excel has multiple sheets, set e.g. "Sheet1"
# =====================================

# Load Excel/CSV file
if file_path.endswith(".csv"):
    df = pd.read_csv(file_path)
else:
    df = pd.read_excel(file_path, sheet_name=sheet_name)

print(" Data loaded successfully!")
print(" Shape:", df.shape)
print(" Preview:", df.head())
print(" Summary Stats:", df.describe(include="all"))

# ================= HEATMAP =================
plt.figure(figsize=(10, 6))
sns.heatmap(df.corr(numeric_only=True), annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap", fontsize=14, fontweight="bold")
plt.show()

# ================= BAR PLOT =================
# Example: average hallucination markers by model
if "model_name" in df.columns and "hallucination_markers" in df.columns:
    plt.figure(figsize=(10, 6))
    sns.barplot(data=df, x="model_name", y="hallucination_markers", estimator="mean", ci="sd")
    plt.title("Average Hallucination Markers by Model", fontsize=14, fontweight="bold")
    plt.xticks(rotation=45)
    plt.show()

# ================= HISTOGRAM =================
plt.figure(figsize=(10, 6))
df.select_dtypes(include="number").hist(bins=20, figsize=(12, 8), color="skyblue", edgecolor="black")
plt.suptitle("Histograms of Numeric Features", fontsize=16, fontweight="bold")
plt.show()

# ================= BOX PLOT =================
plt.figure(figsize=(10, 6))
sns.boxplot(data=df.select_dtypes(include="number"), orient="h", palette="Set2")
plt.title("Boxplot of Numeric Columns", fontsize=14, fontweight="bold")
plt.show()

# ================= PAIRPLOT =================
sns.pairplot(df.select_dtypes(include="number"), diag_kind="kde", corner=True)
plt.suptitle("Pairwise Relationships", y=1.02, fontsize=16, fontweight="bold")
plt.show()
